In [1]:
import cv2
import numpy as np
mser = cv2.MSER_create()

img = cv2.imread("Ipython/Col-dec/Column Testing/In/3.jpg")

vis = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
regions, _ = mser.detectRegions(gray)

hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(vis, hulls, 1, (0, 255, 0))

mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)

recs = []
for contour in hulls:
    max_x = np.max(contour[:,:,0])
    min_x = np.min(contour[:,:,0])
    max_y = np.max(contour[:,:,1])
    min_y = np.min(contour[:,:,1])
    arr = np.array([[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]])
    arr = arr.reshape((-1,1,2))
    recs.append(arr)
    
temp=[]
temp_box=None
old=False
i=0
while i<len(recs):
    if not old:
        temp_box = recs[i]
        old=True
    else:
        t = check(temp_box, recs[i])
        if t:
            temp_box = merge(temp_box,recs[i])
        else:
            temp.append(temp_box)
            temp_box=None
            old=False
    i+=1
temp = np.array(temp)
recs = temp

temp=[]
temp_box=None
old=False
i=0
while i<len(recs):
    if not old:
        temp_box = recs[i]
        old=True
    else:
        t = check(temp_box, recs[i])
        if t:
            temp_box = merge(temp_box,recs[i])
        else:
            temp.append(temp_box)
            temp_box=None
            old=False
    i+=1
temp = np.array(temp)
recs = temp


for rec in recs:
    cv2.drawContours(mask, [rec], -1, (255, 255, 255), -1)

text_only = cv2.bitwise_and(img, img, mask=mask)


NameError: name 'check' is not defined

In [1]:
def submerge(a,b):
    import numpy as np
    if (a[0,0,1]< b[0,0,1] and a[2,0,1]< b[0,0,1]) or (a[0,0,1]> b[2,0,1] and a[2,0,1]> b[2,0,1]) or (a[0,0,0]> b[2,0,0] and a[2,0,0]> b[2,0,0]) or (a[0,0,0]< b[0,0,0] and a[2,0,0]< b[0,0,0]):
        return 1,1
    else:
        min_x = min(a[0,0,0],b[0,0,0])
        max_x = max(a[2,0,0],b[2,0,0])
        min_y = min(a[0,0,1],b[0,0,1])
        max_y = max(a[2,0,1],b[2,0,1])
        new_rec = np.array([[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]])
        new_rec = new_rec.reshape((-1,1,2))
        return 0, new_rec

In [2]:
def check(a,b):
    if ((a[0,0,0] > b[0,0,0] and a[0,0,0]>b[1,0,0]) or (a[1,0,0] < b[0,0,0] and a[1,0,0]<b[1,0,0]) or (a[0,0,1] > b[0,0,1] and a[0,0,1]>b[2,0,1]) or (a[1,0,1] < b[0,0,1] and a[1,0,1]<b[2,0,1])):
        return False
    else:
        return True

In [3]:
def merge(a,b):
    import numpy as np
    temp = np.array(a)
    temp[0,0,0] = min(a[0,0,0],b[0,0,0])
    temp[3,0,0] = temp[0,0,0]
    temp[1,0,0] = max(a[1,0,0],b[1,0,0])
    temp[2,0,0] = temp[1,0,0]
    temp[0,0,1] = min(a[0,0,1],b[0,0,1])
    temp[1,0,1] = temp[0,0,1]
    temp[3,0,1] = max(a[3,0,1],b[3,0,1])
    temp[2,0,1] = temp[3,0,1]
    return temp

In [4]:
def textmask(img):
    import cv2
    import numpy as np
    mser = cv2.MSER_create()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    vis = img.copy()
    regions, _ = mser.detectRegions(gray)
    
    hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
    cv2.polylines(vis, hulls, 1, (0, 255, 0))
    
    mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)
    
    recs = []
    for contour in hulls:
        max_x = np.max(contour[:,:,0])
        min_x = np.min(contour[:,:,0])
        max_y = np.max(contour[:,:,1])
        min_y = np.min(contour[:,:,1])
        arr = np.array([[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]])
        arr = arr.reshape((-1,1,2))
        recs.append(arr)
        
    temp=[]
    temp_box=None
    old=False
    i=0
    while i<len(recs):
        if not old:
            temp_box = recs[i]
            old=True
        else:
            t = check(temp_box, recs[i])
            if t:
                temp_box = merge(temp_box,recs[i])
            else:
                temp.append(temp_box)
                temp_box=None
                old=False
        i+=1
    temp = np.array(temp)
    
    recs = temp
    trig=np.ones((len(temp)))
    print(len(temp))
    
    temp=[]
    i=0
    while i<len(recs):
        if trig[i]==1:
            trig[i]=0
            target = np.array(recs[i])
            j=0
            while j<len(recs):
                if trig[j]==1:
                    out=[]
                    trig[j],out = submerge(target,recs[j])
                    if trig[j]==0:
                        target=out
                j+=1
            temp.append(target)
        i+=1
    recs = np.array(temp)
    print(len(recs))
    print('---------------------------')
        
    
    for rec in recs:
        cv2.drawContours(mask, [rec], -1, (255, 255, 255), -1)

    text_only = cv2.bitwise_and(img, img, mask=mask)
    return vis, text_only

In [5]:
import cv2
import numpy as np
cv2.namedWindow("original", cv2.WINDOW_NORMAL)
cv2.namedWindow("hulls", cv2.WINDOW_NORMAL)
cv2.namedWindow("text", cv2.WINDOW_NORMAL)

cv2.resizeWindow("original", 768, 1024)
cv2.resizeWindow("hulls", 768, 1024)
cv2.resizeWindow("text", 768, 1024)


vc = cv2.VideoCapture(0)

if vc.isOpened():
    rval, frame = vc.read()
else:
    rval = False
    
while rval:
    org = frame.copy()
    hulls, text_only = textmask(frame)
    

    cv2.imshow("original", org)
    cv2.imshow("hulls", hulls)
    cv2.imshow("text", text_only)
    
    rval, frame = vc.read()
    
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break

cv2.destroyWindow("original")
cv2.destroyWindow("hulls")
cv2.destroyWindow("text")

vc.release()

3
3
---------------------------
4
4
---------------------------
5
5
---------------------------
7
6
---------------------------
8
5
---------------------------
9
7
---------------------------
17
7
---------------------------
11
6
---------------------------
16
12
---------------------------
13
9
---------------------------
9
4
---------------------------
7
4
---------------------------
6
5
---------------------------
8
5
---------------------------
8
5
---------------------------
10
7
---------------------------
7
6
---------------------------
7
5
---------------------------
9
7
---------------------------
12
9
---------------------------
13
6
---------------------------
10
3
---------------------------
10
6
---------------------------
7
5
---------------------------
13
8
---------------------------
22
9
---------------------------
13
10
---------------------------
17
8
---------------------------
13
2
---------------------------
11
7
---------------------------
10
6
------------------

60
22
---------------------------
55
31
---------------------------
56
17
---------------------------
62
31
---------------------------
56
22
---------------------------
56
17
---------------------------
44
23
---------------------------
55
30
---------------------------
58
23
---------------------------
67
20
---------------------------
56
18
---------------------------
60
21
---------------------------
61
19
---------------------------
33
15
---------------------------
40
22
---------------------------
36
25
---------------------------
48
29
---------------------------
58
31
---------------------------
60
33
---------------------------
60
28
---------------------------
61
31
---------------------------
47
29
---------------------------
62
38
---------------------------
63
34
---------------------------
54
29
---------------------------
50
29
---------------------------
56
33
---------------------------
52
32
---------------------------
58
36
---------------------------
67
36
--------

73
57
---------------------------
48
30
---------------------------
60
45
---------------------------
20
13
---------------------------
113
61
---------------------------
64
47
---------------------------
42
26
---------------------------
42
26
---------------------------
115
62
---------------------------
114
60
---------------------------
12
7
---------------------------
66
25
---------------------------
45
15
---------------------------
57
24
---------------------------
38
24
---------------------------
68
35
---------------------------
62
22
---------------------------
15
8
---------------------------
41
23
---------------------------
16
8
---------------------------
18
12
---------------------------
66
46
---------------------------
19
12
---------------------------
16
7
---------------------------
30
17
---------------------------
77
32
---------------------------
66
42
---------------------------
41
28
---------------------------
14
7
---------------------------
72
36
----------

110
49
---------------------------
109
56
---------------------------
95
53
---------------------------
101
57
---------------------------
99
48
---------------------------
112
60
---------------------------
100
53
---------------------------
114
52
---------------------------
92
52
---------------------------
119
56
---------------------------
99
50
---------------------------
108
56
---------------------------
107
57
---------------------------
107
54
---------------------------
72
38
---------------------------
100
55
---------------------------
105
57
---------------------------
107
57
---------------------------
107
56
---------------------------
102
47
---------------------------
104
52
---------------------------
113
55
---------------------------
106
56
---------------------------
105
47
---------------------------
113
57
---------------------------
116
51
---------------------------
113
54
---------------------------
119
57
---------------------------
108
51
------------------

## Experiment

In [17]:
import time
import cv2
import numpy as np
mser = cv2.MSER_create()


img = cv2.imread("Ipython/Col-dec/Column Testing/In/3.jpg")
vis = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
regions, _ = mser.detectRegions(gray)



hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(vis, hulls, 1, (0, 255, 0))


recs = []
for contour in hulls:
    max_x = np.max(contour[:,:,0])
    min_x = np.min(contour[:,:,0])
    max_y = np.max(contour[:,:,1])
    min_y = np.min(contour[:,:,1])
    arr = np.array([[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]])
    arr = arr.reshape((-1,1,2))
    recs.append(arr)
    
    
temp=[]
temp_box=None
old=False
i=0
while i<len(recs):
    if not old:
        temp_box = recs[i]
        old=True
    else:
        t = check(temp_box, recs[i])
        if t:
            temp_box = merge(temp_box,recs[i])
        else:
            temp.append(temp_box)
            temp_box=None
            old=False
    i+=1
temp = np.array(temp)
recs = temp


org = np.array(recs)
#ax2 = np.sort(recs, axis=2)
ax0 = np.sort(recs, axis=2)

#orgmask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)
#ax2mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)
ax0mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)



#cv2.namedWindow("original", cv2.WINDOW_NORMAL)
#cv2.resizeWindow("original", 768, 1024)
cv2.namedWindow("axis0", cv2.WINDOW_NORMAL)
cv2.resizeWindow("axis0", 768, 1024)
#cv2.namedWindow("axis2", cv2.WINDOW_NORMAL)
#cv2.resizeWindow("axis2", 768, 1024)

i=0
while i< len(org):
    #cv2.drawContours(orgmask, [org[i]], -1, (255, 255, 255), -1)
    #cv2.drawContours(ax2mask, [ax2[i]], -1, (255, 255, 255), -1)
    cv2.drawContours(ax0mask, [ax0[i]], -1, (255, 255, 255), -1)
    
    #cv2.imshow("original", orgmask)
    cv2.imshow("axis0", ax0mask)
    #cv2.imshow("axis2", ax2mask)
    
    time.sleep(0.01)
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break
        
    i+=1

#cv2.destroyWindow("original")
cv2.destroyWindow("axis0")
#cv2.destroyWindow("axis2")




In [20]:
recs.shape


(879, 4, 1, 2)

In [18]:
import cv2
'''cv2.namedWindow("original", cv2.WINDOW_NORMAL)
cv2.namedWindow("hulls", cv2.WINDOW_NORMAL)
cv2.namedWindow("text", cv2.WINDOW_NORMAL)
cv2.resizeWindow("original", 768, 1024)
cv2.resizeWindow("hulls", 768, 1024)
cv2.resizeWindow("text", 768, 1024)'''

img = cv2.imread("Ipython/Col-dec/Column Testing/New.jpeg")
#rval, frame = vc.read()
#while rval:

hulls, text_only = textmask(img)


#cv2.imshow("original", img)
#cv2.imshow("hulls", hulls)
#cv2.imshow("text", text_only)



        

26
24
---------------------------


In [15]:
text_only.shape

(653, 1160, 3)

In [16]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',text_only)
cv2.waitKey(0)
cv2.destroyAllWindows()